In [3]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import csv
import numpy as np
from ultralytics import YOLO


In [3]:
model = YOLO("yolo11n-pose.pt")

100%|██████████| 5.97M/5.97M [00:00<00:00, 10.8MB/s]


In [ ]:
results = model(source='D:/la-u/ciclo 2025-1/TPI/MODELO/Video pruba/luis2.mp4', show=True, conf=0.3, save=True)




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/138) D:\la-u\ciclo 2025-1\TPI\MODELO\Video pruba\luis2.mp4: 640x384 1 person, 72.5ms
video 1/1 (frame 2/138) D:\la-u\ciclo 2025-1\TPI\MODELO\Video pruba\luis2.mp4: 640x384 1 person, 53.9ms
video 1/1 (frame 3/138) D:\la-u\ciclo 2025-1\TPI\MODELO\Video pruba\luis2.mp4: 640x384 1 person, 53.7ms
video 1/1 (frame 4/138) D:\la-u\ciclo 2025-1\TPI\MODELO\Video pruba\luis2.mp4: 640x384 1 person, 51.7ms
video 1/1 (frame 5/138) D:\la-u\ciclo 2025

: 

In [21]:
# Ruta del video a procesar (modifica la ruta según tu sistema)
video_path = 'D:/la-u/ciclo 2025-1/TPI/MODELO/Video pruba/luis2.mp4'
cap = cv2.VideoCapture(video_path)

# Crear archivo CSV para guardar los keypoints
csv_file = open('keypoints_output_yolo.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['frame', 'persona_index', 'keypoint_index', 'x', 'y', 'confidence'])

frame_count = 0
umbral_conf = 0.2  # Umbral para dibujar keypoints

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Ejecutar la detección en el frame
    results = model(frame, verbose=False)
    
    # Procesar el primer resultado (para el frame actual)
    if len(results) > 0:
        result = results[0]
        
        if result.keypoints is not None:
            for person_idx, person_kpts in enumerate(result.keypoints):
                # Extraer las coordenadas y confianzas
                # person_kpts.xy: coordenadas, person_kpts.conf: confianzas
                if hasattr(person_kpts.xy, 'cpu'):
                    kpts_xy = person_kpts.xy.cpu().numpy()
                else:
                    kpts_xy = person_kpts.xy
                if hasattr(person_kpts.conf, 'cpu'):
                    kpts_conf = person_kpts.conf.cpu().numpy()
                else:
                    kpts_conf = person_kpts.conf

                # Verificar el número de dimensiones y combinar las coordenadas y la confianza
                if kpts_xy.ndim == 3:
                    # Si kpts_xy tiene forma (1, N, 2) y kpts_conf (1, N), concatenamos a lo largo del eje 2
                    keypoints_arr = np.concatenate([kpts_xy, np.expand_dims(kpts_conf, axis=-1)], axis=2)
                    keypoints_arr = keypoints_arr.squeeze(0)  # Queda de forma (N,3)
                elif kpts_xy.ndim == 2:
                    # Si kpts_xy tiene forma (N,2)
                    keypoints_arr = np.hstack([kpts_xy, np.expand_dims(kpts_conf, axis=-1)])
                else:
                    print(f"Formato de keypoints no esperado en frame {frame_count} para persona {person_idx}")
                    continue
                
                # Obtener dimensiones del frame
                height, width, _ = frame.shape

                # Iterar sobre cada keypoint
                for kpt_idx, kpt in enumerate(keypoints_arr):
                    x, y, conf = kpt

                    if conf > umbral_conf:
                        cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)
                    
                    csv_writer.writerow([frame_count, person_idx, kpt_idx, x, y, conf])

    cv2.imshow('Detección de Pose con YOLOv8', frame)
    frame_count += 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
csv_file.close()
print("Procesamiento completado.")

Procesamiento completado.
